# Todos
- [x] Change objective function to minimize the average deivation between actual and ideal assignment frequency between within the assignment pool of each task 
- [x] Update/Record assignment frequency csv
- [ ] Assignments for service, weekly, and monthly assginment slots
- [ ] Weight assignment favorability (scalar per man to augment assignment frequency)
- [ ] Output for html render
- [ ] README
- [ ] tests


In [435]:
import pandas as pd
import os
from collections import OrderedDict
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

In [436]:
# Read eligibility matrix
eligibility_df = pd.read_csv('men.csv')
eligibility_df.set_index('name', inplace=True)

# Read exclusions matrix
exclusions_df = pd.read_csv('exclusions.csv', index_col=0)

# Past assginment frequency
assignments_file = 'previous-assignments.csv'

In [437]:
# helper functions
def is_eligible(person, task) -> bool:
    return eligibility_df.loc[person, task] == 1.0

In [438]:
# List of people and tasks
people = eligibility_df.index

tasks = list(eligibility_df.columns)

# should try to minimize the deviation from the ideal average
ideal_avg = {task: 1/count for task, count in eligibility_df.sum(axis=0).to_dict().items()}

In [439]:
# Initialize or read previous assignments
if os.path.exists(assignments_file):
    previous_assignments_df = pd.read_csv(assignments_file, index_col=0)
else:
    previous_assignments_df = pd.DataFrame(0, index=people, columns=tasks)
    previous_assignments_df['Rounds'] = 0
    
previous_assignments_df['Rounds'] += 1


# Calculate the average assignment frequency for each task
avg_assignments = pd.DataFrame(index=people, columns=tasks)
for person in people:
    for task in tasks:
        avg_assignments.loc[person, task] = previous_assignments_df.loc[person, task] / max(previous_assignments_df.loc[person, 'Rounds'], 1)

# Problem / Objective

In [441]:
# Create the LP problem
# We want to choose asignees so as to Maximize the deviation between their historical mean and the ideal mean
# Over time, we should converge to everyone having the ideal mean
prob = LpProblem("Task_Assignment", LpMaximize)

# Define decision variables

# assignments 
x = LpVariable.dicts("assign", ((person, task) for person in people for task in tasks), cat='Binary')

###
# Objective function: maximize the each persons deviation from the ideal average for each task
###
prob += lpSum(((ideal_avg[task]) - (previous_assignments_df.loc[person, task]/previous_assignments_df.loc[person, 'Rounds'])) * x[(person, task)] for person in people for task in tasks if is_eligible(person, task))


# Constraints

In [443]:
# Only assign eligible people
for person in people:
    for task in tasks:
        if not is_eligible(person, task):
            prob += x[(person, task)] == 0

# Do not assign a person to two excluded tasks
for person in people:
    for task1 in tasks:
            for task2 in tasks:
                if task1 != task2 and exclusions_df.loc[task1, task2] == 1:
                    if is_eligible(person, task1) and is_eligible(person, task1):
                        prob += x[(person, task1)] + x[(person, task2)] <= 1

# Task limit constraints
for person in people:
    prob += lpSum(x[(person, task)] for task in tasks) <= 2

# Task assignment constraints: each task is assigned to exactly one person
for task in tasks:
    prob += lpSum(x[(person, task)] for person in people) == 1

In [444]:
# Solve the problem
result = prob.solve()

# Output the results
assignment = {}
for person in people:
    assigned_tasks = [task for task in tasks if x[(person, task)].varValue == 1]
    if assigned_tasks:
        assignment[person] = assigned_tasks
        for task in assigned_tasks:
            if eligibility_df.loc[person, task] != 1:
                print(f"{person} is NOT eligilbe for {task}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4z/l87fr4g16qb0rxtm42zcxtym0000gn/T/c9251b6f9de34a99b0a5a308a5f1dc28-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/4z/l87fr4g16qb0rxtm42zcxtym0000gn/T/c9251b6f9de34a99b0a5a308a5f1dc28-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3119 COLUMNS
At line 12853 RHS
At line 15968 BOUNDS
At line 16912 ENDATA
Problem MODEL has 3114 rows, 943 columns and 7445 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.76737 - 0.00 seconds
Cgl0002I 541 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 566 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 599 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 459 strengthened rows, 0 subs

# Output

In [446]:

# Output the results
assignments = pd.DataFrame(0, index=people, columns=tasks)
for person in people:
    for task in tasks:
        assignments.loc[person, task] = 1 if x[(person, task)].varValue == 1.0 else 0

# Update the previous assignments with the current ones
for person in people:
    for task in tasks:
        if assignments.loc[person, task]:
            previous_assignments_df.loc[person, task] += 1

# Save the updated assignments to a CSV file
previous_assignments_df.to_csv(assignments_file)


In [447]:

schedule = OrderedDict({task : [person for person in assignment.keys() if task in assignment[person]][0] for task in tasks})

for key, value in list(schedule.items()):
    print(f"{key}: {value}")
    
assignments.loc[(assignments!=0).any(axis=1)].shape

first_opening_prayer: Love, Chris
first_lesson: Nunn, Jeff
first_song_leader: Vinson, Joe
announcements: McAlister, Brady
song_leader: Schiffman, Bobby
scripture_reading: McAlister, Grady
opening_prayer: McAnear, Justus
closing_prayer: Gray, Allen
table_lead_bread: Taylor, Keith
table_lead_cup: Earp, Wyatt
table_aid_bread: Melvyn, Joe
table_aid_cup: McAnear, Jaxan
lesson: Byers, Austin
wednesday_announcements: Taylor, Keith
wednesday_song_leader: Byers, Austin
wednesday_opening_prayer: Purcell, Tripp
wednesday_lesson: Tipton, Sam
wednesday_closing_prayer: Gray, Allen
lords_supper_prep: Hight, Ben
usher: Hight, Ben
alt_usher: Nunn, Sam
security: Warren, David
sound_board_operator: Smiley, Justin


(19, 23)